In [4]:
# Importing the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Splitting the Data into Training and Testing Sets

### Step 1: Reading the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [5]:
# Reading the CSV file from the Resources folder into a Pandas DataFrame
lending_data = pd.read_csv(
    "lending_data.csv")

# Reviewing the DataFrame
lending_data.head(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
5,10100.0,7.438,50600,0.407115,4,1,20600,0
6,10300.0,7.490,51100,0.412916,4,1,21100,0
7,8800.0,6.857,45100,0.334812,3,0,15100,0
8,9300.0,7.096,47400,0.367089,3,0,17400,0
9,9700.0,7.248,48800,0.385246,4,0,18800,0


### Step 2: Creating the labels set (`y`)  from the “loan_status” column, and then creating the features (`X`) DataFrame from the remaining columns.

In [7]:
# Separating the data into labels and features
X = lending_data.copy()

# Separating the y variable, the labels
y = lending_data["loan_status"].values.reshape(-1, 1)

# Separating the X variable, the features
X.drop("loan_status", axis=1, inplace=True)

In [8]:
# Reviewing the y variable Series
y[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [9]:
# Reviewing the X variable DataFrame
X[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Checking the balance of the labels variable (`y`) by using the `value_counts` function.

In [11]:
# Checking the balance of our target values
lending_data["loan_status"].value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Splitting the data into training and testing datasets by using `train_test_split`.

In [12]:
# Importing the train_test_learn module
from sklearn.model_selection import train_test_split

# Spliting the data using train_test_split
# Assigning a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Creating a Logistic Regression Model with the Original Data

###  Step 1: Fitting a logistic regression model by using the training data (`X_train` and `y_train`).

In [38]:
# Importing the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiating the Logistic Regression model
# Assigning a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fitting the model using training data
classifier.fit(X_train, y_train)

/Users/sumaiyanathani/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=1)

### Step 2: Saving the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [15]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)

### Step 3: Evaluating the model’s performance by doing the following:

* Calculating the accuracy score of the model.

* Generating a confusion matrix.

* Printing the classification report.

In [16]:
# Printing the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9520479254722232

In [20]:
# Generating a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,18663,102
Actual 1,56,563


In [19]:
# Printing the classification report for the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answering the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicts the healthy loan better (higher precision and recall scores) than the high-risk loan labels (lower precision and recall scores). The overall accuracy of the model is great (0.95), but the model doesn't do well in prediciting the high-risk loan labels because of unbalanced data in this category.

---

## Predicting a Logistic Regression Model with Resampled Training Data

### Step 1: Using the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Making sure to confirm that the labels have an equal number of data points. 

In [23]:
# Importing the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler
                               
# Instantiating the random oversampler model
# # Assigning a random_state parameter of 1 to the model
oversample = RandomOverSampler(sampling_strategy='minority')

# Fitting the original training data to the random_oversampler model
X_over, y_over = oversample.fit_resample(X_train, y_train)

In [33]:
# Counting the distinct values of the resampled labels data
np.unique(y_over, return_counts=True)


(array([0, 1]), array([56271, 56271]))

### Step 2: Using the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [41]:
# Instantiating the Logistic Regression model
# Assigning a random_state parameter of 1 to the model
classifier_over = LogisticRegression(solver='lbfgs', random_state=1)

# Fitting the model using the resampled training data
classifier.fit(X_over, y_over)

# Making a prediction using the testing data
predictions_over = classifier.predict(X_test)

### Step 3: Evaluating the model’s performance by doing the following:

* Calculating the accuracy score of the model.

* Generating a confusion matrix.

* Printing the classification report.

In [42]:
# Printing the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, predictions_over)

0.9936781215845847

In [43]:
# Generating a confusion matrix for the model
cm_over = confusion_matrix(y_test, predictions_over)
cm_df_over = pd.DataFrame(
    cm_over, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df_over

,Predicted 0,Predicted 1
Actual 0,18649,116
Actual 1,4,615


In [37]:
# Printing the classification report for the model
print(classification_report(y_test, predictions_over))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answering the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic model fits better with the oversampled data than with the original data. The recall score for both labels is 0.99 which is very high which means that the model is able to identify all individuals who fall into this category along with some individuals that belong to the healthy loan category. But the precision metric for high-risk loan lables is slightly low which means that the model is misidentifying some of the healthy loan labels as high-risk loan labels. For predicting the healthy loan labels, the model has a slighly higher precision (1.00) than recall score (0.99). The balanced accuracy score is quite high (0.99) which indicates that the model fits well with the oversampled data overall.